## Introducción.

El dataset trabajado cuenta con un registro de 50 millones de manos jugadas de "BlackJack" usando un simulador. Cada registro cuenta con 12 columnas, de las cuales se valerá nuestra red neuronal para predecir la acción a tomar:
- Pedir una carta (HIT).
- No pedir otra carta (STAND).

## Parte 1 - Análisis de la base de datos.

### 1. Descripción de columnas.

Nuestro dataset cuenta con muchas columnas que terminaremos reduciendo a las que consideramos mas relevantes. Sin embargo, resumiremos la utilidad de la totalidad de ellas:
- "Shoe id": Es un identificador del mazo utilizado (del 1 al 8).
- "Cards remaining": Son las cartas restantes del mazo al empezar la ronda.
- "Dealer up": Es la carta visible del "dealer" al empezar la ronda.
- "Initial hand": Son las 2 cartas dadas al jugador.
- "Dealer final": Cartas con las que termina el dealer al final de la ronda.
- "Dealer final value":  Es el valor de las cartas mencionadas en la columna anterior.
- "Player final": Al igual que dealer final, son las cartas con las que termina el jugador.
- "Player final value": La suma del valor de las cartas del jugador.
- "Actions taken": Es la acción que toma el el jugador, "H" o "S".
- "Run count": Identificador de la ronda a jugar.

### 2. Analisis de correlaciones.

Por un lado, consideramos varias columnas como poco impactantes, como por ejemplo:
- Shoe id: El identificador del mazo no afecta a la estrategia del juego.
- Run count: No afecta el numero de ronda jugada con la decisión a tomar.
- Cards remaining: A menos que se esté haciendo un conteo de cartas (lo cual es un analisis muy avanzado), no suele impactar en la decisión final.

Ahora bien, hay columnas que representan correlacionves clave para la decisión a tomar:
- Initial hand => Actions taken: La correlación en este caso es alta(positiva o negativamente). Las manos de 17 a 21 correlacionan con STAND, mientras que las menores a 11 se correlacionan con HIT
- Dealer up => Actions taken: La correlación es alta y negativa, ya que si el dealer tiene una mano "alta", el jugador tenderá a hacer HIT, y si la mano del dealer es "baja", el jugador hará STAND.



In [ ]:
# Importar librerias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#Me interesa: ; dealer_up; initial_hand; actions_taken
columnas = ['dealer_up','initial_hand','actions_taken']
df = pd.read_csv("csv_reducido",usecols=columnas)
df.head(5)

,dealer_up,initial_hand,actions_taken
0,10,"[6, 3]","[['H', 'H']]"
1,10,"[10, 5]",[['R']]
2,2,"[6, 8]",[['S']]
3,8,"[7, 3]",[['D']]
4,6,"[10, 5]",[['S']]


### 3 - Análisis de factibilidad.

Consideramos el dataset elegido como idóneo para entrar una red neuronal, y sobretodo con las columnas seleccionadas.
Dichas columnas son predictoras naturales de la acción que se tomará. A su vez, esta columna de decisión es una variable objetivo binaria y categórica, ideal para la tarea de clasificación.

Dicho esto, lo que en última medida predecirá la red, es la acción ideal a tomar, una vez presentada la carta del dealer así como las del jugador.

### 4 - Datos atípicos y limpieza.

Al regirnos a las reglas mismas del Blackjack, es imposible encontrar Outliers por definición.


### 5 - Transformaciones preliminares.

Los datos presentados originalmente no servirán de mucho, puesto que tenemos listas que la red neuronal no entenderá, así como strings, por lo que debemos convertir initial_hand y actions_taken en variables que la red sí pueda comprender.
En este caso sumaremos los valores de las cartas de initial_hand para llegar a el valor total del jugador. Con respecto a actions_taken, nos quedaremos solamente con HIT y STAND, que son las 2 acciones mas comunes, y les daremos un valor binario, ideal para la predicción.

In [147]:
# Sumar toda la initial hand para que sea un valor
df['hand_value'] = df['initial_hand'].apply(lambda x: sum(eval(x)) if isinstance(x, str) else sum(x))

# Solo me interesa la primera accion, si Hitteo, o si hizo Stay
acciones_validas = ['H','S']


def get_accion_valida(actions):
    try:
        actions = eval(actions)[0] if isinstance(actions, str) else actions[0]
    except Exception:
        return None
    for action in actions:
        if action in acciones_validas:
            return action
    return None

# Recorro la columna actions taken y aplico la funcion para filtrar y le asigno un valor binario
df['first_action'] = df['actions_taken'].apply(get_accion_valida)
df = df[df['first_action'].notnull()]
df['action'] = df['first_action'].map({'S':0,'H':1})

# Creo ya las columnas con todos valores listos para interpretarse
inputs = df[['dealer_up','hand_value']]
outputs = df[['action']]
df_preview = pd.concat([inputs, outputs], axis=1)
print(df_preview.head())



   dealer_up  hand_value  action
0         10           9       1
2          2          14       0
4          6          15       0
5         11           9       1
8          6          12       0


El siguiente paso será normalizar los datos que tenemos. 
Aplicaremos el **MinMaxScaler** de **sklearn**, que lo que hará será convertir nuestro hand value a valores de entre 0 y 1, pero teniendo en cuenta que los valores originales iban de entre 4 a 21.
Tambien escalaremos el dealer_up de la misma forma, pero teniendo en cuenta en este caso que los valores originales iban de 2 a 11. Es decir, la escala que se utiliza para convertir los valores entre 0 y 1 será diferente en ambos casos.

In [148]:

scaler = MinMaxScaler()
X = scaler.fit_transform(df_preview[['dealer_up', 'hand_value']])

# La salida ya es binaria, asi que no hace falta normalizarla

#Inputs normalizadas
df_inputs = pd.DataFrame(X, columns=['dealer_up_normalizado', 'hand_value_normalizado'])

#Concateno con la action
df_normalizado =  pd.concat([df_inputs, df_preview['action'].reset_index(drop=True)], axis=1)

df_normalizado.head()



,dealer_up_normalizado,hand_value_normalizado,action
0,0.888889,0.294118,1
1,0.000000,0.588235,0
2,0.444444,0.647059,0
3,1.000000,0.294118,1
4,0.444444,0.470588,0


## PARTE 2 - DESARROLLO DE LA RED NEURONAL

Ya con los datos preparados vamos a comenzar haciendo un test de presicion con la red sin entrenar aplicando Forward Propagation

In [149]:
# Con 100.000 datos, usamos el 20% como conjunto de prueba
X_train, X_test, Y_train, Y_test = train_test_split(df_inputs, outputs, test_size=0.2)

# Guardo el numero de registros de entrenamiento
n = X_train.shape[0]

# Construimos una red neuronal simple con pesos y sesgos
w_hidden = np.random.rand(3, 2)  # Tenemos 2 entradas, son las dimensiones de la matriz
w_output = np.random.rand(1, 3)

b_hidden = np.random.rand(3, 1)
b_output = np.random.rand(1, 1)

# Defino las funciones activación
relu = lambda x: np.maximum(x, 0)  # Convierte todos los valores negativos en 0
logistic = lambda x: 1 / (1 + np.exp(-x))  # Es la función sigmoide, convierte todos los valores de salida a un número entre 1 y 0

# Defino la función forward_prop, que es la fórmula tradicional de Descenso de Gradiente Estocástico
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden  # Aplico la fórmula con la entrada
    A1 = relu(Z1)                 # Primera función activación
    Z2 = w_output @ A1 + b_output # Hago la fórmula con la salida anterior activada
    A2 = logistic(Z2)             # Salida activada
    return Z1, A1, Z2, A2

# Calculo de precisión
test_predictions = forward_prop(X_test.T)[3]  # Me interesa solo la capa de salida (A2)

test_comparisons = np.equal(test_predictions >= 0.5, Y_test.values.reshape(1, -1)).astype(int) # Devuelve true si la prediccion fue mayor a 0.5, y asegura q los valores de Y_test tengan la misma forma
accuracy = np.mean(test_comparisons)
print("ACCURACY: ", accuracy)



ACCURACY:  0.4378405778735268


Como vemos en el resultado de la acurracy, nuestra red no entrenada acierta menos de la mitad de las veces, que no es ideal, por lo que ahora vamos a hacer **Backpropagation**, aplicando **descenso de gradiente estocastico** para entrenar la red

In [151]:
# En primer lugar vamos a declarar una taza de aprendizaje para nuestra red
L = 0.01

# Luego, derivamos las funciones de activacion, que nos serviran para calcular derivadas parciales a la hora de definir BackPropagation
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2


# Definimos Backpropagation
def backward_prop(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = d_logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Ejecutamos el Descenso de gradiente
for i in range(100_000):
    # Elijo aleatoriamente uno de los datos dedicados a entrenamiento (el otro 80%)
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train.iloc[idx].values.T # Forma (2,1)
    Y_sample = Y_train.iloc[idx].values.reshape(1,1) # Forma (1,1)

    # Paso los datos seleccionados por la red neuronal de forma aleatoria
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # Aplico retropropagacion para distribuir errores y devolver pendientes para los pesos y los sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # Actualizo los pesos y los sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

# Calculo de precision actualizado
test_predictions = forward_prop(X_test.T)[3]
test_comparisons = np.equal(test_predictions >= 0.5, Y_test.values.reshape(1, -1)).astype(int)
accuracy = np.mean(test_comparisons)
print("ACCURACY: ", accuracy)




ACCURACY:  0.9036877455328856
